In [1]:
import tokenize_uk
import pymorphy2
from gensim.models import KeyedVectors
from gensim.models.keyedvectors import Word2VecKeyedVectors
from typing import List, Dict
from fuzzyset import FuzzySet

In [2]:
embeddings_file = "./embeddings/ubercorpus.cased.tokenized.word2vec.300d"

In [3]:
word2vec = KeyedVectors.load_word2vec_format(embeddings_file)

In [4]:
w2v_vocab = FuzzySet(word2vec.vocab)

In [5]:
class Preprocessor:
    morph_analyzer = pymorphy2.MorphAnalyzer(lang='uk')
    
    def __init__(self, word2vec: Word2VecKeyedVectors, 
                 stop_words: List[str] = list(), 
                 fixed_word_dict: Dict[str, List[str]] = dict()):
        self.word2vec = word2vec
        self.approximate_matcher = FuzzySet(word2vec.vocab)
        self.stop_words = stop_words
        self.fixed_word_dict = fixed_word_dict
        
        
    def preprocess_texts(self, texts: List[str]):
        results = [self.preprocess_text(text) for text in texts]
        return results
        
    def preprocess_text(self, text: str) -> List[str]:
        result: List[str] = []
        tokens = tokenize_uk.tokenize_words(text)
        for token in tokens:
            if self.__should_token_be_included(token):
                if token in self.word2vec:
                    result.append(token)
                else:
                    fixed_word = self.__fix_word(token)
                    if fixed_word is not None:
                        result.append(fixed_word)
            
                
        
                
        return result
    
    def __should_token_be_included(self, token: str) -> bool:
        if token in self.stop_words:
            return False
        
        if self.__check_punctuation(token) is False:
            return False
        
        
        return True
    
    def __check_punctuation(self, token: str) -> bool:
        parse = self.morph_analyzer.parse(token)[0]
        
        if 'PNCT' in parse.tag:
            return False
        
        #TODO: add more 
        
        return True
    
    
    def __fix_word(self, word: str):
        if word in self.fixed_word_dict:
            return self.fixed_word_dict[word]
        
        candidate = self.approximate_matcher.get(word)
        if candidate is not None and len(candidate) > 0:
            fixed_word = candidate[0][1]
            self.fixed_word_dict[word] = fixed_word
            return fixed_word
        
        return None

In [6]:
preprocess = Preprocessor(word2vec)

In [7]:
preprocess.preprocess_texts(["Куди мені йт?"])

[['Куди', 'мені', 'йти']]

### Load data

In [8]:
import pandas as pd

In [9]:
df = pd.read_csv("data/reviews.csv")

In [10]:
df.head()

,Unnamed: 0,title,pos_text,neg_text,ratingValue,bestRating
0,0,Ідеально для ділової поїздки!,Ідеально для ділової поїздки! Господар зустрів...,NaN,10.0,10.0
1,1,"Затишний, чистий номер з усіма зручностями.","Затишний, чистий номер з усіма зручностями. Чу...","При бронюванні вказала час прибуття о 7 ранку,...",9.2,10.0
2,2,Все сподобалося. Рекомендую,"Чисто, тихо, комфортно. Зустрів і провів приєм...",На барі кава тільки 3 в 1. Хотілося звичайної ...,9.6,10.0
3,3,"Зручне розташування,чудовий вигляд з вікна,в н...","Зручне розташування,чудовий вигляд з вікна,в н...",NaN,10.0,10.0
4,4,"Все чудово: 9,9 балів!","Нові апартаменти на останньому поверсі ЖК, біл...","Немає терміналу для оплати кредиткою, тільки г...",10.0,10.0


In [11]:
preprocessed_data_dict = {
    'title': [],
    'pos': [],
    'neg': [],
    'ratingValue': [],
    'bestRating': [],
    'rate' : []
}

In [12]:
df.iloc[0]['title']

'Ідеально для ділової поїздки!'

In [13]:
preprocess.preprocess_text(df.iloc[0]['neg_text'])

['Nan']

In [14]:
for i in range(0, len(df)):
    preprocessed_data_dict['title'].append(preprocess.preprocess_text(df.iloc[i]['title']))
    preprocessed_data_dict['pos'].append(preprocess.preprocess_text(df.iloc[i]['pos_text']))
    preprocessed_data_dict['neg'].append(preprocess.preprocess_text(df.iloc[i]['neg_text']))
    
    ratingValue = df.iloc[i]['ratingValue']
    preprocessed_data_dict['ratingValue'].append(ratingValue)
    preprocessed_data_dict['bestRating'].append(df.iloc[i]['bestRating'])
    preprocessed_data_dict['rate'].append(-1)
    if ratingValue > 6 and ratingValue < 9:
        preprocessed_data_dict['rate'][-1] = 0
    elif ratingValue >= 9:
        preprocessed_data_dict['rate'][-1] = 1
        
#     preprocessed_data_dict['rate'].append(rate)

In [15]:
preprocessed_data_dict['ratingValue'][-1] >= 9

True

In [16]:
len(preprocessed_data_dict['ratingValue'])

15993

In [17]:
len(preprocessed_data_dict['rate'])

15993

In [18]:
preprocessed_df = pd.DataFrame.from_dict(preprocessed_data_dict)
preprocessed_df.head()

,title,pos,neg,ratingValue,bestRating,rate
0,"[Ідеально, для, ділової, поїздки]","[Ідеально, для, ділової, поїздки, Господар, зу...",[Nan],10.0,10.0,1
1,"[Затишний, чистий, номер, з, усіма, зручностями]","[Затишний, чистий, номер, з, усіма, зручностям...","[При, бронюванні, вказала, час, прибуття, о, 7...",9.2,10.0,1
2,"[Все, сподобалося, Рекомендую]","[Чисто, тихо, комфортно, Зустрів, і, провів, п...","[На, барі, кава, тільки, 3, в, 1, Хотілося, зв...",9.6,10.0,1
3,"[Зручне, розташування, чудовий, вигляд, з, вік...","[Зручне, розташування, чудовий, вигляд, з, вік...",[Nan],10.0,10.0,1
4,"[Все, чудово, 9,9, балів]","[Нові, апартаменти, на, останньому, поверсі, Ж...","[Немає, терміналу, для, оплати, кредиткою, тіл...",10.0,10.0,1


In [19]:
preprocessed_df.iloc[0]['title']

['Ідеально', 'для', 'ділової', 'поїздки']

In [20]:
preprocessed_df.to_csv("data/reviews_preprocessed.csv")